# SQL Challenge: Online Store Database Querying

This notebook demonstrates a simple SQL challenge using an online store database scenario. We will:  
1. Create the necessary tables.
2. Insert sample data.
3. Perform various queries to extract insights.


## 1. Create Tables
We will create the following tables:  
- **Customers**: Stores customer information.  
- **Orders**: Stores order information linked to customers.  
- **Products**: Stores product details.  
- **OrderDetails**: Stores details of each order.


In [1]:
import sqlite3
import pandas as pd

In [3]:
# Create a new SQLite database in memory

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [5]:
# 1. Create Tables    #let's create the Customers table
cursor.execute('''
    CREATE TABLE Customers (
        customer_id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        email TEXT NOT NULL,
        join_date DATE NOT NULL
    );
''')

In [7]:
#Let's create the Orders table

cursor.execute('''
    CREATE TABLE Orders (
        order_id INTEGER PRIMARY KEY,
        customer_id INTEGER,
        order_date DATE NOT NULL,
        total_amount DECIMAL NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
    );
''')

In [9]:
#Let's create the Products table

cursor.execute('''
    CREATE TABLE Products (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT NOT NULL,
        price DECIMAL NOT NULL
    );
''')


In [11]:
#Let's create the OrderDetails table

cursor.execute('''
    CREATE TABLE OrderDetails (
        order_detail_id INTEGER PRIMARY KEY,
        order_id INTEGER,
        product_id INTEGER,
        quantity INTEGER NOT NULL,
        price DECIMAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES Orders(order_id),
        FOREIGN KEY (product_id) REFERENCES Products(product_id)
    );
''')

### 2. Insert Sample Data
#### We will insert at least 5 rows of sample data into each table.

In [14]:
# Insert Values into Customers table

cursor.executemany('''
    INSERT INTO Customers (name, email, join_date) VALUES (?, ?, ?);
''', [
    ('Alice', 'alice@example.com', '2025-01-01'),
    ('Bob', 'bob@example.com', '2025-01-15'),
    ('Charlie', 'charlie@example.com', '2024-12-20'),
    ('Diana', 'diana@example.com', '2025-02-10'),
    ('Eve', 'eve@example.com', '2024-11-25')
])

In [16]:
#Insert Values into the Products table

cursor.executemany('''
    INSERT INTO Products (product_name, category, price) VALUES (?, ?, ?);
''', [
    ('Laptop', 'Electronics', 1000.00),
    ('Headphones', 'Electronics', 100.00),
    ('Coffee Maker', 'Home Appliances', 50.00),
    ('Smartphone', 'Electronics', 800.00),
    ('Book', 'Stationery', 20.00)
])


In [18]:
#Insert Values into the Orders Table

cursor.executemany('''
    INSERT INTO Orders (customer_id, order_date, total_amount) VALUES (?, ?, ?);
''', [
    (1, '2025-01-02', 1100.00),
    (2, '2025-01-16', 100.00),
    (3, '2025-01-20', 850.00),
    (1, '2025-02-05', 20.00),
    (4, '2025-02-12', 50.00)
])

In [20]:
#Insert Values into the OrderDetails Table

cursor.executemany('''
    INSERT INTO OrderDetails (order_id, product_id, quantity, price) VALUES (?, ?, ?, ?);
''', [
    (1, 1, 1, 1000.00),
    (1, 2, 1, 100.00),
    (2, 2, 1, 100.00),
    (3, 4, 1, 800.00),
    (3, 2, 1, 50.00),
    (4, 5, 1, 20.00),
    (5, 3, 1, 50.00)
])

In [22]:
conn.commit()

### 3. Queries
### Customers who joined in the last 30 days


In [25]:
query_a = '''
    SELECT * FROM Customers
    WHERE join_date >= DATE('now', '-30 days');
'''

In [27]:
df_a = pd.read_sql_query(query_a, conn)
df_a

,customer_id,name,email,join_date
0,4,Diana,diana@example.com,2025-02-10


### Total sales amount for each customer

In [30]:
query_b = '''
    SELECT C.name, SUM(O.total_amount) AS total_spent
    FROM Customers C
    LEFT JOIN Orders O ON C.customer_id = O.customer_id
    GROUP BY C.customer_id;
'''

In [32]:
df_b = pd.read_sql_query(query_b, conn)
df_b

,name,total_spent
0,Alice,1120.0
1,Bob,100.0
2,Charlie,850.0
3,Diana,50.0
4,Eve,NaN


### Top 3 best-selling products by quantity


In [35]:
query_c = '''
    SELECT P.product_name, SUM(OD.quantity) AS total_sold
    FROM Products P
    JOIN OrderDetails OD ON P.product_id = OD.product_id
    GROUP BY P.product_id
    ORDER BY total_sold DESC
    LIMIT 3;
'''

In [37]:
df_c = pd.read_sql_query(query_c, conn)
df_c

,product_name,total_sold
0,Headphones,3
1,Book,1
2,Smartphone,1


### Order details for Alice

In [40]:
query_d = '''
    SELECT O.order_id, P.product_name, OD.quantity, OD.price, (OD.quantity * OD.price) AS total_amount
    FROM Orders O
    JOIN OrderDetails OD ON O.order_id = OD.order_id
    JOIN Products P ON OD.product_id = P.product_id
    WHERE O.customer_id = 1;
'''


In [42]:
df_d = pd.read_sql_query(query_d, conn)
df_d

,order_id,product_name,quantity,price,total_amount
0,1,Laptop,1,1000,1000
1,1,Headphones,1,100,100
2,4,Book,1,20,20


### Customers who have never placed an order

In [45]:
query_e = '''
    SELECT C.name
    FROM Customers C
    LEFT JOIN Orders O ON C.customer_id = O.customer_id
    WHERE O.order_id IS NULL;
'''

In [47]:
df_e = pd.read_sql_query(query_e, conn)
df_e

,name
0,Eve


In [49]:
## Closing the Connection
conn.close()